YouTubeチャンネル移行

In [ ]:
import os
import csv
import pickle
import google.oauth2.credentials
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
import time
from googleapiclient.errors import HttpError

class YouTubeChannelMigrationTool:

  def __init__(self, client_secret_json):
    self.CLIENT_SECRETS_FILE = client_secret_json
    self.SCOPES = ['https://www.googleapis.com/auth/youtubepartner', 'https://www.googleapis.com/auth/youtube', 'https://www.googleapis.com/auth/youtube.force-ssl']
    self.API_SERVICE_NAME = 'youtube'
    self.API_VERSION = 'v3'
    print("Authenticate old account")
    self.service_old = self.get_authenticated_service(pickleFileName="CREDENTIALS_PICKLE_FILE_OLD")
    print("Authenticate new account")
    self.service_new = self.get_authenticated_service(pickleFileName="CREDENTIALS_PICKLE_FILE_NEW")
    self.courseId = None

  # Google Auth Init
  def get_authenticated_service(self, pickleFileName="CREDENTIALS_PICKLE_FILE"):
    if os.path.exists(pickleFileName):
        with open(pickleFileName, 'rb') as f:
            credentials = pickle.load(f)
    else:
        flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(self.CLIENT_SECRETS_FILE, self.SCOPES)
        credentials = flow.run_console()
        with open(pickleFileName, 'wb') as f:
            pickle.dump(credentials, f)
    return googleapiclient.discovery.build(self.API_SERVICE_NAME, self.API_VERSION, credentials=credentials)

  # Get Liked Videos
  def get_liked_videos_old(self, nextPageToken=None, output=[]):
    request = self.service_old.playlistItems().list(
        part="snippet",
        playlistId="LL",
        pageToken=nextPageToken
    ).execute()
    for info in request["items"]:
      output.append(info["snippet"]["resourceId"]["videoId"])
    try:
      nextPageToken = request["nextPageToken"]
    except:
      return output
    else:
      return self.get_liked_videos_old(nextPageToken, output)

  # Get Liked Videos
  def get_liked_videos_new(self, nextPageToken=None, output=[]):
    request = self.service_new.playlistItems().list(
        part="snippet",
        playlistId="LL",
        pageToken=nextPageToken
    ).execute()
    for info in request["items"]:
      output.append(info["snippet"]["resourceId"]["videoId"])
    try:
      nextPageToken = request["nextPageToken"]
    except:
      return output
    else:
      return self.get_liked_videos_new(nextPageToken, output)

  # Set Liked Videos
  def set_liked_videos(self, videoId):
    request_body = {
          "snippet": {
            "playlistId": "LL",
            "resourceId": {
              "kind": "youtube#video",
              "videoId": videoId
            }
          }
        }
    request = self.service_new.playlistItems().insert(
        part="snippet",
        body=request_body
    ).execute()
    return request

  # Get Subscriptions Old Account
  def get_subscriptions_old(self, nextPageToken=None, output=[]):
    request = self.service_old.subscriptions().list(
        part="snippet",
        mine="true",
        pageToken=nextPageToken
    ).execute()
    for info in request["items"]:
      output.append(info["snippet"]["resourceId"]["channelId"])
    try:
      nextPageToken = request["nextPageToken"]
    except:
      return output
    else:
      return self.get_subscriptions_old(nextPageToken, output)

  # Get Subscriptions New Account
  def get_subscriptions_new(self, nextPageToken=None, output=[]):
    request = self.service_new.subscriptions().list(
        part="snippet",
        mine="true",
        pageToken=nextPageToken
    ).execute()
    for info in request["items"]:
      output.append(info["snippet"]["resourceId"]["channelId"])
    try:
      nextPageToken = request["nextPageToken"]
    except:
      return output
    else:
      return self.get_subscriptions_new(nextPageToken, output)

  # Set Subscriptions (Subscribe Channel)
  def set_subscriptions(self, channelId):
    request_body = {
          "snippet": {
            "resourceId": {
              "channelId": channelId
            }
          }
        }
    request = self.service_new.subscriptions().insert(
        part="snippet",
        body=request_body
    ).execute()
    return request

GoogleのAPIを有効化するために、Google API Consoleからプロジェクトを作成し、Oauth Desktopクライアントを作成 (JSONで認証情報をダウンロードしてくる)すると同時に、YouTube Data API V3を有効化しておく

In [ ]:
YTCMT = YouTubeChannelMigrationTool("./client_secret_hogehoge.json")

高評価した動画リストを移行

In [ ]:
liked_videos_old = YTCMT.get_liked_videos_old()
liked_videos_new = YTCMT.get_liked_videos_new()
liked_videos = list(set(liked_videos_old) - set(liked_videos_new))
print(len(liked_videos))
for id, video in enumerate(reversed(liked_videos[:-134])):
  for _ in range(3):
    try:
      YTCMT.set_liked_videos(video)
    except HttpError as e:
      print(id, video, e)
      if e.resp.status == 404:
        break
      elif e.resp.status == 403:
        time.sleep(120)
      else:
        time.sleep(10)
    else:
      break
  else:
    raise ValueError
  print(id)
  time.sleep(5)

登録チャンネルリストを移行

In [ ]:
subscriptions_old = YTCMT.get_subscriptions_old()
subscriptions_new = YTCMT.get_subscriptions_new()
subscriptions = list(set(subscriptions_old) - set(subscriptions_new))
print(len(subscriptions))
for id, channel in enumerate(subscriptions):
  for _ in range(3):
    try:
      YTCMT.set_subscriptions(channel)
    except HttpError as e:
      print(id, channel, e)
      if e.resp.status == 400:
        break
      elif e.resp.status == 403:
        time.sleep(120)
      else:
        time.sleep(10)
    else:
      break
  else:
    raise ValueError
  print(id)
  time.sleep(5)